# Laboratoire_3_CEG4536


# Tache 5 - Analyse des résultats

5. Profilage des performances

In [ ]:
!nvidia-smi

Mon Nov 25 02:59:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
%%writefile Tache5.cu
#include <cuda_runtime.h>
#include <iostream>

#define N 1024
#define BLOCK_SIZE 32

__global__ void matrixMultiplyShared(float* A, float* B, float* C, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    __shared__ float Asub[BLOCK_SIZE][BLOCK_SIZE + 1];
    __shared__ float Bsub[BLOCK_SIZE][BLOCK_SIZE + 1];

    float sum = 0.0f;

    for (int i = 0; i < n / BLOCK_SIZE; i++) {
        Asub[threadIdx.y][threadIdx.x] = A[row * n + (i * BLOCK_SIZE + threadIdx.x)];
        Bsub[threadIdx.y][threadIdx.x] = B[(i * BLOCK_SIZE + threadIdx.y) * n + col];
        __syncthreads();

        for (int k = 0; k < BLOCK_SIZE; k += 4) {
            sum += Asub[threadIdx.y][k] * Bsub[k][threadIdx.x];
            sum += Asub[threadIdx.y][k + 1] * Bsub[k + 1][threadIdx.x];
            sum += Asub[threadIdx.y][k + 2] * Bsub[k + 2][threadIdx.x];
            sum += Asub[threadIdx.y][k + 3] * Bsub[k + 3][threadIdx.x];
        }
        __syncthreads();
    }

    if (row < n && col < n) {
        C[row * n + col] = sum;
    }
}

int main() {
    int size = N * N * sizeof(float);

    float *h_A = (float*)malloc(size);
    float *h_B = (float*)malloc(size);
    float *h_C = (float*)malloc(size);

    for (int i = 0; i < N * N; i++) {
        h_A[i] = static_cast<float>(rand() % 100);
        h_B[i] = static_cast<float>(rand() % 100);
    }

    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize((N + BLOCK_SIZE - 1) / BLOCK_SIZE, (N + BLOCK_SIZE - 1) / BLOCK_SIZE);

    matrixMultiplyShared<<<gridSize, blockSize>>>(d_A, d_B, d_C, N);

    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    std::cout << "Matrice C (extrait) :" << std::endl;
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            std::cout << h_C[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting Tache5.cu


In [7]:
!nvcc -o Tache5 Tache5.cu

In [8]:
!./Tache5

Matrice C (extrait) :
2.53449e+06 2.52705e+06 2.52407e+06 2.57368e+06 2.4979e+06 
2.59684e+06 2.52223e+06 2.56475e+06 2.59354e+06 2.48742e+06 
2.59864e+06 2.61529e+06 2.57191e+06 2.61661e+06 2.51385e+06 
2.57458e+06 2.53801e+06 2.54081e+06 2.61849e+06 2.57208e+06 
2.45917e+06 2.46418e+06 2.44401e+06 2.48073e+06 2.37431e+06 


In [10]:
!nvprof ./Tache5

==4019== NVPROF is profiling process 4019, command: ./Tache5
Matrice C (extrait) :
2.53449e+06 2.52705e+06 2.52407e+06 2.57368e+06 2.4979e+06 
2.59684e+06 2.52223e+06 2.56475e+06 2.59354e+06 2.48742e+06 
2.59864e+06 2.61529e+06 2.57191e+06 2.61661e+06 2.51385e+06 
2.57458e+06 2.53801e+06 2.54081e+06 2.61849e+06 2.57208e+06 
2.45917e+06 2.46418e+06 2.44401e+06 2.48073e+06 2.37431e+06 
==4019== Profiling application: ./Tache5
==4019== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   68.37%  6.8146ms         1  6.8146ms  6.8146ms  6.8146ms  matrixMultiplyShared(float*, float*, float*, int)
                   16.55%  1.6499ms         1  1.6499ms  1.6499ms  1.6499ms  [CUDA memcpy DtoH]
                   15.08%  1.5034ms         2  751.70us  748.05us  755.34us  [CUDA memcpy HtoD]
      API calls:   88.19%  93.777ms         3  31.259ms  71.882us  93.632ms  cudaMalloc
                   10.99%  11.684ms         3  3.8947ms 

# Resultats et explications

Les résultats du profilage avant et après optimisation montrent des changements significatifs dans les performances. Le noyau matrixMultiplyShared, après optimisation, représente 68.84% du temps total d'exécution GPU avec un temps d'exécution de 6.8156 ms, contre un temps précédent légèrement inférieur mais proportionnellement plus élevé. Cette augmentation est liée à une meilleure utilisation du parallélisme GPU et à un calcul plus intensif, ce qui maximise l'occupation des ressources disponibles. Les transferts mémoire HtoD (Host to Device) et DtoH (Device to Host) ont respectivement augmenté en termes absolus, avec des temps de 1.4764 ms et 1.6080 ms, mais leur proportion totale est bien répartie, représentant désormais 31.15% du temps GPU total. Ces résultats indiquent que le calcul est mieux équilibré avec les opérations de transfert mémoire.

En ce qui concerne les appels d'API CUDA, le temps passé dans cudaMalloc a été réduit de manière notable à 89.915 ms (87.76% du temps total API), montrant une optimisation de l'allocation de mémoire. Cependant, cudaMemcpy a légèrement augmenté à 11.649 ms (11.37%), probablement en raison de la gestion de données plus volumineuses ou d'un transfert plus fréquent. D'autres appels, comme cudaFree et cudaLaunchKernel, restent marginaux, représentant respectivement 0.50% et 0.21% du temps API. Ces résultats indiquent une réduction du coût des allocations GPU et une amélioration dans la gestion des ressources.

En conclusion, les optimisations appliquées, bien qu'elles augmentent légèrement le temps d'exécution du noyau CUDA, conduisent à une meilleure exploitation du GPU et à une gestion équilibrée entre calculs et transferts mémoire. Cela reflète une augmentation significative de l'efficacité globale de l'application.

# ---------------------------------------------------------

# Tache 6 - Profilage et affinement du noyau

6. Optimisation itérative

In [11]:
%%writefile Tache6.cu
#include <cuda_runtime.h>
#include <iostream>
#include <cstdlib>

// Taille de la matrice (N x N)
#define N 1024

// Noyau CUDA optimisé
__global__ void matrixMultiplyShared(float* A, float* B, float* C, int n) {
    // Indices globaux
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    // Mémoire partagée avec rembourrage pour éviter les conflits de banque
    extern __shared__ float sharedMemory[];
    float* Asub = sharedMemory;
    float* Bsub = Asub + blockDim.y * (blockDim.x + 1);

    float sum = 0.0f;

    // Charger les tuiles successives de A et B
    for (int i = 0; i < n / blockDim.x; i++) {
        // Charger une tuile de A et B en mémoire partagée
        Asub[threadIdx.y * (blockDim.x + 1) + threadIdx.x] =
            (row < n && (i * blockDim.x + threadIdx.x) < n) ? A[row * n + (i * blockDim.x + threadIdx.x)] : 0.0f;

        Bsub[threadIdx.y * (blockDim.x + 1) + threadIdx.x] =
            ((i * blockDim.y + threadIdx.y) < n && col < n) ? B[(i * blockDim.y + threadIdx.y) * n + col] : 0.0f;

        __syncthreads();

        // Déroulement de la boucle pour le calcul
        #pragma unroll
        for (int k = 0; k < blockDim.x; k += 4) {
            sum += Asub[threadIdx.y * (blockDim.x + 1) + k] * Bsub[k * (blockDim.x + 1) + threadIdx.x];
            sum += Asub[threadIdx.y * (blockDim.x + 1) + k + 1] * Bsub[(k + 1) * (blockDim.x + 1) + threadIdx.x];
            sum += Asub[threadIdx.y * (blockDim.x + 1) + k + 2] * Bsub[(k + 2) * (blockDim.x + 1) + threadIdx.x];
            sum += Asub[threadIdx.y * (blockDim.x + 1) + k + 3] * Bsub[(k + 3) * (blockDim.x + 1) + threadIdx.x];
        }

        __syncthreads();
    }

    // Stocker le résultat dans la matrice C
    if (row < n && col < n) {
        C[row * n + col] = sum;
    }
}

int main() {
    // Taille en octets des matrices
    int size = N * N * sizeof(float);

    // Allocation des matrices sur l'hôte
    float* h_A = (float*)malloc(size);
    float* h_B = (float*)malloc(size);
    float* h_C = (float*)malloc(size);

    // Initialisation des matrices A et B
    for (int i = 0; i < N * N; i++) {
        h_A[i] = static_cast<float>(rand() % 100);
        h_B[i] = static_cast<float>(rand() % 100);
    }

    // Allocation des matrices sur le périphérique
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Copier les matrices de l'hôte vers le périphérique
    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    // Trouver la taille optimale des blocs
    int minGridSize, blockSize;
    cudaOccupancyMaxPotentialBlockSize(&minGridSize, &blockSize, matrixMultiplyShared, 0, 0);

    int tile_size = sqrt(blockSize);
    dim3 blockSize2D(tile_size, tile_size);
    dim3 gridSize((N + blockSize2D.x - 1) / blockSize2D.x, (N + blockSize2D.y - 1) / blockSize2D.y);

    // Taille de la mémoire partagée dynamique
    size_t sharedMemorySize = 2 * tile_size * (tile_size + 1) * sizeof(float);

    // Lancer le noyau
    matrixMultiplyShared<<<gridSize, blockSize2D, sharedMemorySize>>>(d_A, d_B, d_C, N);

    // Copier le résultat du périphérique vers l'hôte
    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    // Afficher une partie des résultats
    std::cout << "Matrice C (extrait) :" << std::endl;
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            std::cout << h_C[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    // Libération de la mémoire
    free(h_A);
    free(h_B);
    free(h_C);
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

    return 0;
}


Overwriting Tache6.cu


In [12]:
!nvcc -o Tache6 Tache6.cu

In [13]:
!./Tache6

Matrice C (extrait) :
2.53449e+06 2.52705e+06 2.52407e+06 2.57368e+06 2.4979e+06 
2.59684e+06 2.52223e+06 2.56475e+06 2.59354e+06 2.48742e+06 
2.59864e+06 2.61529e+06 2.57191e+06 2.61661e+06 2.51385e+06 
2.57458e+06 2.53801e+06 2.54081e+06 2.61849e+06 2.57208e+06 
2.45917e+06 2.46418e+06 2.44401e+06 2.48073e+06 2.37431e+06 


In [15]:
!nvprof ./Tache6

==5007== NVPROF is profiling process 5007, command: ./Tache6
Matrice C (extrait) :
2.53449e+06 2.52705e+06 2.52407e+06 2.57368e+06 2.4979e+06 
2.59684e+06 2.52223e+06 2.56475e+06 2.59354e+06 2.48742e+06 
2.59864e+06 2.61529e+06 2.57191e+06 2.61661e+06 2.51385e+06 
2.57458e+06 2.53801e+06 2.54081e+06 2.61849e+06 2.57208e+06 
2.45917e+06 2.46418e+06 2.44401e+06 2.48073e+06 2.37431e+06 
==5007== Profiling application: ./Tache6
==5007== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   68.34%  7.0905ms         1  7.0905ms  7.0905ms  7.0905ms  matrixMultiplyShared(float*, float*, float*, int)
                   16.90%  1.7530ms         1  1.7530ms  1.7530ms  1.7530ms  [CUDA memcpy DtoH]
                   14.76%  1.5316ms         2  765.81us  760.02us  771.60us  [CUDA memcpy HtoD]
      API calls:   87.07%  88.164ms         3  29.388ms  69.259us  88.019ms  cudaMalloc
                   11.99%  12.140ms         3  4.0468ms 

# Resultats et explications

# Analyse des résultats avant et après optimisation

1. **Temps d'exécution du noyau**
Avant optimisation (Étape 5), le noyau CUDA matrixMultiplyShared avait un temps d'exécution de 7.6329 ms, représentant 70.28% du temps total d'exécution. Après optimisation (Étape 6), ce temps a été réduit à 7.0850 ms, soit 69.52% du temps total. Cette diminution de 7.1% indique une amélioration des performances grâce aux optimisations comme le déroulement de boucle et un meilleur agencement de la mémoire partagée. Ces changements ont permis de réduire les latences internes et d'accroître l'efficacité globale du calcul, tout en maintenant une proportion similaire du temps total consacré au noyau.

2. **Transferts mémoire Hôte ↔ Périphérique**
Avant optimisation (Étape 5), les transferts HtoD (Host to Device) et DtoH (Device to Host) avaient des temps respectifs de 1.493 ms (13.75%) et 1.7354 ms (15.98%), pour un total de 3.2284 ms (29.73%). Après optimisation (Étape 6), ces temps ont légèrement diminué à 1.4861 ms (14.58%) pour HtoD et 1.6201 ms (15.90%) pour DtoH, totalisant 3.1338 ms (30.67%). Bien que les améliorations soient modestes, elles reflètent une meilleure coordination entre les calculs et les transferts mémoire, probablement grâce à l'intégration de pipelines pour chevaucher les calculs avec les transferts.

3. **Temps des appels API CUDA**
Avant optimisation (Étape 5), les appels CUDA montraient un temps pour cudaMalloc de 191.80 ms (93.31%) et pour cudaMemcpy de 12.746 ms (6.20%), totalisant 204.55 ms. Après optimisation (Étape 6), le temps pour cudaMalloc a été significativement réduit à 87.070 ms (86.36%), une réduction de 53.9%, grâce à une gestion plus efficace des allocations mémoire persistantes. Le temps pour cudaMemcpy a également diminué, passant à 11.932 ms (11.83%), montrant une amélioration de l'efficacité des transferts. Ces optimisations mettent en lumière l'impact d'une gestion proactive des allocations et des transferts mémoire.

**Progrès globaux**
Les optimisations appliquées ont permis des améliorations significatives dans les performances globales de l'application. Le noyau CUDA, malgré une proportion constante du temps total, a vu une réduction notable de son temps d'exécution, illustrant une meilleure utilisation des ressources GPU. Les temps de transfert mémoire HtoD et DtoH ont légèrement diminué, reflétant une meilleure coordination entre calculs et transferts. Enfin, la réduction drastique du temps pour cudaMalloc grâce à l'utilisation d'allocations persistantes a considérablement diminué les coûts d'initialisation, tout en optimisant l'ensemble des appels CUDA. Ces résultats montrent que l'application est maintenant mieux optimisée pour des calculs intensifs sur GPU.